In [2]:
# USE TRIE for encoding dictionary and INCLUDE comments
# possile alphabet = [a-zA-Z0-9?! ,.;:\n]

In [3]:
# encoding
# use arary format trie similar to aho-corasick implementation
# total number of possible alphabets is 70 -> default dictionary is size of 70
# With char dictionary, each column of each char can be found by char itself as index

def get_default_trie(data):
    # possible alphabets = [a-zA-Z0-9 !?,.;:\n]
    pos_chars = [i for i in range(ord('a'), ord('z')+1)] + \
                [i for i in range(ord('A'), ord('Z')+1)] + \
                [i for i in range(ord('0'), ord('9')+1)] + \
                [ord('!'), ord('?'), ord(' '), ord(','), ord('.'), ord(':'), ord(';'), ord('\n')]
    available_char_dict = {}
    for i, c in enumerate(pos_chars):
        available_char_dict[chr(c)] = i

    # initialize each cell with -1 to compare whether the given pattern does exist in the dictionary
    default_trie = [[-1]*70 for _ in range(len(data))]
    return default_trie, available_char_dict

In [5]:
def encoding(data):
     # build default trie and char dictionary
    trie, char_dic = get_default_trie(data)
    out = []

    present_state = 0
    data_state = 1

    for i in range(len(data)):#c in data:
        c = data[i]
        char = char_dic[c]
        # if there is no matching char in the trie
        if trie[present_state][char] == -1:
            trie[present_state][char] = data_state
            out.append((present_state, c))

            # reset for search
            present_state = 0
            data_state += 1
        # if matching prefix char exists in the trie
        else:
            present_state = trie[present_state][char]
            
    # if pattern exists in the trie
    if present_state >0:
        out.append((present_state, ''))
            
    return out

In [6]:
def decoding(data):
    d_out = []
    for (n, c) in data:
        d_out.append(c if n == 0 else d_out[n-1] + c)
    return ''.join(d_out)

In [7]:
def read_file(file_name):
    with open(file_name, 'r') as f:
        data = ''.join(f.readlines())
    return data

In [8]:
def compress(origin_file, compressed_file):
    import struct

    data = read_file(origin_file)
    encoded_data = encoding(data)

    with open(compressed_file, 'wb') as compressed_file:
        for i, (idx, ch) in enumerate(encoded_data):
            save_char = ch.encode() if len(ch) > 0 else b'\x00'
            
            # when sequence is less than 256 (2^8 = 1 byte)
            # 2 byte = 1 byte of idx and 1 byte of ascii code for char
            if i <= 255:
                data = struct.pack('Bc', idx, save_char)
                compressed_file.write(data) 
            # when sequence is less than 65536 ((2^8)^2 = 2 byte)
            # 3 byte = 2 byte of idx and 1 byte of ascii code for char
            elif i <= 65535:
                data = struct.pack('Hc', idx, save_char)
                compressed_file.write(data) 
            # when sequence is less than 16777216 (((2^8)^2)^2 = 2 byte)
            # 4 byte = 3 byte of idx and 1 byte of ascii code for char
            elif i <= 16777215:
                data = struct.pack('Ic', idx, save_char)
                compressed_file.write(b''.join([data[0:3], data[4:]])) 
            # when sequence is greater than 16777216 (((2^8)^2)^2 = 2 byte)
            # 5 byte = 4 byte of idx and 1 byte of ascii code for char
            else:
                data = struct.pack('Ic', idx, save_char)
                compressed_file.write(data)            

In [9]:
def decompress(compressed_file, decompressed_file):
        import struct 
        bin_file = open(compressed_file, 'rb')
        bin_type = {2: 'Bc', 3: 'Hc', 4: 'Ic', 5: 'Ic'}
        
        encoded_data = []
        seq = 0
        
        while True:
            # when sequence is less than 256 (2^8 = 1 byte)
            # 2 byte = 1 byte of idx and 1 byte of ascii code for char
            if seq <= 255:
                bin_size = 2
            # when sequence is less than 65536 ((2^8)^2 = 2 byte)
            # 3 byte = 2 byte of idx and 1 byte of ascii code for char
            elif seq <= 65535:
                bin_size = 3
            # when sequence is less than 16777216 (((2^8)^2)^2 = 2 byte)
            # 4 byte = 3 byte of idx and 1 byte of ascii code for char
            elif seq <= 16777215:
                bin_size = 4
            # when sequence is greater than 16777216 (((2^8)^2)^2 = 2 byte)
            # 5 byte = 4 byte of idx and 1 byte of ascii code for char
            else:
                bin_size = 5
            
            # read by bin_size determined by the sequence index
            binary = bin_file.read(bin_size)
            
            # check the end of the file
            if binary == b'': 
                break 
                
            # when bin_size 4, need to add \x00 as 4th byte because I've removed the last byte for compression
            if bin_size ==4:
                encoded_data.append(struct.unpack(bin_type[bin_size], b''.join([binary[0:3], b'\x00', binary[3:]])))
            else:
                encoded_data.append(struct.unpack(bin_type[bin_size], binary))
            
            seq += 1
        
        if encoded_data[-1][1] == b'\x00':
            encoded_data[-1] = (encoded_data[-1][0], b'')
            
        encoded_data = [(d[0], d[1].decode()) for d in encoded_data]
        decoded_data = decoding(encoded_data)
        
        with open(decompressed_file, 'w') as decompressed_file:
            decompressed_file.write(decoded_data)

In [ ]:
%%time

data = ['ABBCBCABABCAABCAAB', 'BABAABRRRA', 'AAAAAAAAA']

for d in data:
    print(d)
    encoded = encoding(d)
    print(encoded)
#     print(decoding(encoded))
    print('***')

In [22]:
# encoded = encoding(read_file('./infile.txt'))
# encoded = encoding(read_file('./sample1/titles_condition.csv'))
# encoded = encoding(read_file('./sample2/aladdin.txt'))
# encoded = encoding(read_file('./sample3/aesops_fables.txt'))

In [27]:
# compress('./infile.txt', 'compressed_test')
# compress('./sample1/titles_condition.csv', './sample1/compressed_titles')
# compress('./sample2/aladdin.txt', './sample2/compressed_aladdin')
# compress('./sample3/aesops_fables.txt', './sample3/compressed_aesops')

1.64 s ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit

# decompress('./compressed_test', 'decompressed')
# decompress('./sample1/compressed_titles', './sample1/decompressed_titles')
# decompress('./sample2/compressed_aladdin', './sample2/decompressed_aladdin')
decompress('./sample3/compressed_aesops', './sample3/decompressed_aesopes')

325 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
compress('./infile.txt', 'compressed_test')
decompress('./compressed_test', 'decompressed')

In [ ]:
compress('./sample1/titles_condition.csv', './sample1/compressed_titles')
decompress('./sample1/compressed_titles', './sample1/decompressed_titles')

In [ ]:
compress('./sample2/aladdin.txt', './sample2/compressed_aladdin')
decompress('./sample2/compressed_aladdin', './sample2/decompressed_aladdin')

In [ ]:
compress('./sample3/aesops_fables.txt', './sample3/compressed_aesops')
decompress('./sample3/compressed_aesops', './sample3/decompressed_aesopes')